# Module 5 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Solving Normal Form Games

In [1]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> List[Tuple]:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return the empty List (`[]`).


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

<a id="stateify_game"></a>
## stateify_game

`stateify_game` makes the `game` into a state for State Space Search. To do so, it adds two elements to the tuples in `game`: the strategy numbers for each player. The new cell layout is: `(P1 strategy, P2 strategy, P1 payoff, P2 payoff)`. The function returns a new game board and does not modify the original. **Used by**: [solve_game](#solve_game)

* **game**: the game board to stateify

**returns** `List[List[Tuple]]`: the new game board as a state

In [3]:
def stateify_game(game: List[List[Tuple]]) -> List[List[Tuple]]:
    game_state = []
    for row in range(len(game)):
        game_state.append([(row, col, elem[0], elem[1]) for col, elem in enumerate(game[row])])
    return game_state

In [4]:
# assertions/unit tests
test_game =  [[( -5, -5), (-1,-10)],
        [(-10, -1), (-2, -2)]]
actual_game = stateify_game(test_game)
assert actual_game == [[(0, 0, -5, -5), (0, 1, -1, -10)], 
                       [(1, 0, -10, -1), (1, 1, -2, -2)]]

test_game = [[(0, 0), (0, 1)],
             [(1, 0), (1, 1)]]
actual_game = stateify_game(test_game)
assert actual_game == [[(0, 0, 0, 0), (0, 1, 0, 1)], [(1, 0, 1, 0), (1, 1, 1, 1)]]

test_game = []
actual_game = stateify_game(test_game)
assert actual_game == test_game

<a id="find_dominated_strats"></a>
## find_dominated_strats

`find_dominated_strats` returns a list of row and column indices for dominated strategies present in `curr_state`. The function compares all pairs of strategies and checks whether one strategy is greater or strictly greater (depending on the `weak` parameter) than the other. If one strategy dominates the other, the index of the row or column is added to a list, and both lists of row and column indices are returned at the end. It is important to note that a weakly dominated strategy requires that all values of the dominating strategy must be at least greater than the dominated strategy, with *at least* one value strictly greater (i.e., two identical strategies cannot dominate each other). **Used by**: [is_terminal](#is_terminal), [successors](#successors)

* **curr_state**: current state in which to find dominated strategies
* **weak**: parameter for strong/weak domination criteria

**returns** `Tuple[List[Tuple]]`: the lists of dominated row and dominated column indices

In [5]:
def find_dominated_strats(curr_state: List[List[Tuple]], weak: bool) -> Tuple[List[Tuple]]:
    dominated_rows, dominated_cols = [], []
    num_rows, num_cols = len(curr_state), len(curr_state[0]) if curr_state else 0
    for i in range(num_rows):
        for j in range(num_rows):
            if i != j:
                if weak: 
                    dominated = all(curr_state[j][k][2] >= curr_state[i][k][2] for k in range(num_cols)) and \
                                any(curr_state[j][k][2] > curr_state[i][k][2] for k in range(num_cols))
                else: dominated = all(curr_state[j][k][2] > curr_state[i][k][2] for k in range(num_cols))
                if dominated: dominated_rows.append(i)
    for i in range(num_cols):
        for j in range(num_cols):
            if i != j:
                if weak: 
                    dominated = all(curr_state[k][j][3] >= curr_state[k][i][3] for k in range(num_rows)) and \
                                any(curr_state[k][j][3] > curr_state[k][i][3] for k in range(num_rows))
                else: dominated = all(curr_state[k][j][3] > curr_state[k][i][3] for k in range(num_rows))
                if dominated: dominated_cols.append(i)
    return dominated_rows, dominated_cols

In [6]:
# assertions/unit tests
test_game = [[( -5, -5), (-1,-10)],
        [(-10, -1), (-2, -2)]]
test_game_state = stateify_game(test_game)
actual_rows, actual_cols = find_dominated_strats(test_game_state, False)
assert actual_rows == [1] and actual_cols == [1]

test_game = [[(2, 0), (1, 1)], 
             [(2, 2), (3, 3)]]
test_game_state = stateify_game(test_game)
actual_rows, actual_cols = find_dominated_strats(test_game_state, True)
assert actual_rows == [0] and actual_cols == [0]

test_game_state = []
actual_rows, actual_cols = find_dominated_strats(test_game_state, True)
assert actual_rows == actual_cols == []

<a id="is_terminal"></a>
## is_terminal

`is_terminal` checks whether a state is a terminal state - this is determined by checking whether the state is either a singular strategy (one tuple) or if there are no dominated strategies present in the current game state. If there are no strategies, the state is added to the `terminal` list. Otherwise, it is added to the `solutions` list. Both lists are returned at the end of the function, along with a boolean value `terminal_check` that notes whether either list changed (whether a failure or success state was added). **Uses**: [find_dominated_strats](#find_dominated_strats). **Used by**: [solve_game](#solve_game)

* **curr_state**: state to check terminality of
* **terminals**: list of terminal (failure) states thus far
* **solutions**: list of success states thus far
* **weak**: parameter for strong/weak domination criteria

**returns** `Tuple[List[Tuple]]`: a tuple of `(terminals, solutions, terminal_check)`

In [7]:
def is_terminal(curr_state: List[List[Tuple]], terminals: List[Tuple], solutions: List[Tuple], weak: bool) \
                -> Tuple[List[Tuple]]:
    terminal_check = False
    if len(curr_state)== 1 and len(curr_state[0]) == 1:
        curr_strategy = (curr_state[0][0][0], curr_state[0][0][1])
        if curr_strategy not in solutions: solutions.append(curr_strategy)
        terminal_check = True
    else:
        dominated_rows, dominated_cols = find_dominated_strats(curr_state, weak)
        if not (dominated_rows or dominated_cols) and curr_state not in terminals:
            terminals.append(curr_state)
            terminal_check = True
    return terminals, solutions, terminal_check

In [8]:
# assertions/unit tests
test_state = [[(0, 0, -5, -5)]]
terminals, solutions, terminal_check = is_terminal(test_state, [], [], False)
assert solutions

test_state = [[(0, 0, 1, 2), (0, 1, 2, 1)],
              [(1, 0, 2, 1), (1, 1, 1, 2)]]
terminals, solutions, terminal_check = is_terminal(test_state, [], [], False)
assert terminals

test_state = []
terminals, solutions, terminal_check = is_terminal(test_state, [], [], False)
assert not (terminals[0] or solutions)

<a id="successors"></a>
## successors

`successors` generates a list of child states from `curr_state`, which are found after eliminating any dominated strategies in `curr_state`. Dominated strategies are found from `find_dominated_strats`, and a deepcopy of the original state is then modified to remove each index of row/columns in the lists returned from `find_dominated_strats`. **Uses**: [find_dominated_strats](#find_dominated_strats). **Used by**: [solve_game](#solve_game).

* **curr_state**: the current board to find successors of
* **weak**: parameter for strong/weak domination criteria

**returns** `List[List[Tuple]]`: a list of child boards from a single elimination of `curr_state`

In [9]:
def successors(curr_state: List[List[Tuple]], weak: bool) -> List[List[Tuple]]:
    children = []
    dominated_rows, dominated_cols = find_dominated_strats(curr_state, weak)
    for dominated_row in dominated_rows:
        new_state = deepcopy(curr_state)
        del new_state[dominated_row]
        children.append(new_state)
    for dominated_col in dominated_cols:
        new_state = deepcopy(curr_state)
        for row in new_state:
            del row[dominated_col]
        children.append(new_state)
    return children

In [10]:
# assertions/unit tests
test_game = [[( -5, -5), (-1,-10)],
             [(-10, -1), (-2, -2)]]
test_game_state = stateify_game(test_game)
actual_children = successors(test_game_state, False)
assert actual_children[0] == [[(0, 0, -5, -5), (0, 1, -1, -10)]]
assert actual_children[1] == [[(0, 0, -5, -5)], [(1, 0, -10, -1)]]

test_game_state = [[(0, 0, -5, -5), (0, 1, -1, -10)]]
actual_children = successors(test_game_state, False)
assert actual_children == [[[(0, 0, -5, -5)]]]

test_game_state = [[(0, 0, -5, -5)], [(1, 0, -10, -1)]]
actual_children = successors(test_game_state, False)
assert actual_children == [[[(0, 0, -5, -5)]]]

<a id="solve_game"></a>
## solve_game

`solve_game` takes a normal form representation of a two-player game and finds all Nash equilibrium points in the game through Successive Elimination of Dominated Strategies. In this method, we find dominated strategies in the normal form game:
* A strategy `a` is *strongly* dominated by strategy `b` if every payoff of strategy `b` is greater than that of strategy `a`, regardless of the other player's strategy.
* A strategy `c` is *weakly* dominated by strategy `d` if every payoff of strategy `d` is at least as great as that of strategy `c`, with at least one payoff being strictly greater. 

The optional `weak` parameter (default `False`) specifies whether to solve for Successive Elimination of Strongly Dominated or Weakly Dominated Strategies. 

The algorithm is a modified version of State Space Search. In this implementation, states are game boards, with elements in the board having the following form: `(P1 Strategy, P2 Strategy, P1 Payoff, P2 Payoff)`. Transitions are eliminations of dominated strategies, and children are the result of a single elimination from the parent (if any exist). Each state represents a game board with one or more dominated strategies eliminated. The algorithm slowly eliminates dominated strategies until either:
* there is only one strategy pair left, which is a success state
* there are only non-dominated strategies left, which is a failure state

The final list of success states is returned at the end. **Uses**: [stateify_game](#stateify_game), [is_terminal](#is_terminal), [successors](#successors).

* **game**: the normal form game on which to perform the SEDS algorithm
* **weak**: optional parameter for strong/weak domination criteria

**returns** `List[Tuple]`: a list of success strategies, `[(P1 Strategy, P2 Strategy)]`.

In [11]:
def solve_game(game: List[List[Tuple]], weak:bool=False) -> List[Tuple]:
    game_state = stateify_game(game)
    frontier, solutions, terminals = [game_state], [], []
    while frontier:
        curr_state = frontier.pop()
        terminals, solutions, terminal_check = is_terminal(curr_state, terminals, solutions, weak)
        if not terminal_check:
            children = successors(curr_state, weak)
            for child in children:
                if child not in frontier and child not in solutions and child not in terminals: 
                    frontier.append(child)
    return solutions

## Additional Directions

Create three games as described and according to the following:

1. Your games must be created and solved "by hand".
2. The strategy pairs must **not** be on the main diagonal (0, 0), (1, 1), or (2, 2). And the solution cannot be the same for both Game 1 and Game 2.
3. Make sure you fill out the Markdown ("?") with your game as well as the solution ("?").
4. Remember, **do not return the payoff**, return the strategy indices (a list of them).

## Before you code...

Solve the following game by hand using SEDS and weakly dominated strategies. 
The game has three (pure) Nash Equilibriums. 
You should find all of them.
This will help you think about what you need to implement to make the algorithm work.
**Hint**: You will need State Space Search from Module 1 and SEDS from Module 5 to get the full algorithm to work.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions**: (0, 2), (2, 1)

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 2/2 | 3/3 | 2/1 |
|1  | 1/2 | 2/3 | 1/1 |
|2  | 1/3 | 1/2 | 0/1 |

**Solution:** (0, 1)

In [12]:
test_game_1 = [[(2, 2), (3, 3), (2, 1)],
               [(1, 2), (2, 3), (1, 1)],
               [(1, 3), (1, 2), (0, 1)]]

solution = solve_game(test_game_1)

In [13]:
assert sorted(solution) == sorted([(0, 1)]) # insert your solution from above.

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 2/0 | 2/2 | 2/3 |
|1  | 2/3 | 1/2 | 1/3 |
|2  | 2/1 | 2/1 | 0/1 |

**Solution:** (0, 2)

In [14]:
test_game_2 = [[(2, 0), (2, 2), (2, 3)], 
               [(2, 3), (1, 2), (1, 3)], 
               [(2, 1), (2, 1), (0, 1)]]

strong_solution = solve_game( test_game_2)
weak_solution = solve_game( test_game_2, weak=True)

In [15]:
assert strong_solution == []
assert sorted(weak_solution) == sorted([(0, 2)]) # insert your solution from above.

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/1 | 2/2 | 3/3 |
|1  | 2/2 | 3/3 | 1/2 |
|2  | 3/3 | 2/1 | 1/1 |

**Solution:** None

In [16]:
test_game_3 = [
    [(1, 1), (2, 2), (3, 3)],
    [(2, 2), (3, 3), (1, 2)],
    [(3, 3), (2, 1), (1, 1)]]

strong_solution = solve_game( test_game_3)
weak_solution = solve_game( test_game_3, weak=True)

In [17]:
assert strong_solution == []
assert weak_solution == []

### Test Game 4. Multiple Equilibria

You solve the following game by hand, above.
Now use your code to solve it.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions:** (0, 2), (2, 1)

In [18]:
test_game_4 = [
[(1, 0), (3, 1), (1, 1)],
[(1, 1), (3, 0), (0, 1)],
[(2, 2), (3, 3), (0, 2)]]

strong_solution = solve_game( test_game_4)
weak_solution = solve_game( test_game_4, weak=True)

In [19]:
assert strong_solution == []
assert sorted(weak_solution) == sorted([(0, 2), (2, 1)]) # put solution here

## Before You Submit...

1. Did you provide output exactly as requested? **Don't forget to fill out the Markdown tables with your games**.
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.

## Comments

Although the instructions said there are 3 Nash equilibrium points, I could only find 2 from the process described in class. My algorithm matches this, so I assumed this is an oversight, rather than a mistake in my algorithm.